# Time Evolution

In [1]:
import numpy as np

from qilisdk.analog import Schedule, X, Y, Z
from qilisdk.backends import QutipBackend
from qilisdk.core import ket, tensor_prod
from qilisdk.functionals import TimeEvolution

T = 10  # Total evolution time
dt = 0.1  # Time-step
steps = np.linspace(0, T, int(T / dt))
nqubits = 1

# Define two Hamiltonians for the simulation
H1 = sum(X(i) for i in range(nqubits))
H2 = sum(Z(i) for i in range(nqubits))

# Create a schedule for the time evolution
schedule = Schedule(
    T,
    dt,
    hamiltonians={"h1": H1, "h2": H2},
    schedule={t: {"h1": 1 - steps[t] / T, "h2": steps[t] / T} for t in range(len(steps))},
)

# Prepare an initial state (equal superposition)
state = tensor_prod([(ket(0) + ket(1)).unit() for _ in range(nqubits)]).unit()

# Perform time evolution on the CUDA backend with observables to monitor
time_evolution = TimeEvolution(
    schedule=schedule, initial_state=state, observables=[Z(0), X(0), Y(0)], store_intermediate_results=True
)

backend = QutipBackend()
results = backend.execute(time_evolution)
print("Time Evolution Results:", results)

Time Evolution Results: TimeEvolutionResult(
  final_expected_values=array([ 0.99426889, -0.03403984,  0.10134429]),
  expected_values=array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 6.31847574e-09,  1.00000000e+00,  1.22854620e-05],
       [ 2.05610973e-04,  9.99997828e-01,  2.07392438e-03],
       [ 1.01183010e-03,  9.99980848e-01,  6.10576876e-03],
       [ 2.77816943e-03,  9.99924730e-01,  1.19505316e-02],
       [ 5.81553140e-03,  9.99794905e-01,  1.93991410e-02],
       [ 1.03795510e-02,  9.99548169e-01,  2.82085466e-02],
       [ 1.66675105e-02,  9.99136560e-01,  3.80569196e-02],
       [ 2.48063566e-02,  9.98508954e-01,  4.86262688e-02],
       [ 3.48574234e-02,  9.97613969e-01,  5.95930331e-02],
       [ 4.68193048e-02,  9.96402464e-01,  7.06405214e-02],
       [ 6.06328224e-02,  9.94830382e-01,  8.14626994e-02],
       [ 7.61866869e-02,  9.92861062e-01,  9.17741808e-02],
       [ 9.33252160e-02,  9.90468542e-01,  1.01303860e-01],
       [ 1.11848622e-01, 